## Задание 2
Данные Immunotherapy.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import chi2_contingency, ttest_ind

In [2]:
df = pd.read_excel('data/Immunotherapy.xlsx')
df

,sex,age,Time,Number_of_Warts,Type,Area,induration_diameter,Result_of_Treatment
0,1,22,2.25,14,3,51,50,1
1,1,15,3.00,2,3,900,70,1
2,1,16,10.50,2,1,100,25,1
3,1,27,4.50,9,3,80,30,1
4,1,20,8.00,6,1,45,8,1
...,...,...,...,...,...,...,...,...
85,1,40,5.50,8,3,69,5,1
86,1,38,7.50,8,2,56,45,1
87,1,46,11.50,4,1,91,25,0
88,1,32,12.00,9,1,43,50,0


### Пол
Так как данные о поле дискретны, будем проверять гипозу "результат лечиния зависит от пола" с помощью критерия хи-квадрат для теста независимости. Не забудем проверить условие применения метода.

In [3]:
def test_dependency(column1, column2, alpha):
    table = np.array([[len(df[(df[column1] == c1) & (df[column2] == c2)]) 
                              for c1 in df[column1].unique()] 
                              for c2 in df[column2].unique()])
    # Application boundaries
    assert np.all(table >= 6)
    
    chi2, pval, dof, expected = chi2_contingency(table)
    return(pval > alpha)

In [4]:
alpha = 0.05
test_dependency('sex', 'Result_of_Treatment', alpha)

True

**Вывод**: результат лечения не зависит от пола.

### Возраст

Будем использовать тот же критерий, но разобьем пациентов на группы по возрасту.

In [5]:
alpha = 0.05
df['age_group'] = np.digitize(df.age, [15, 33, 42, 60])
test_dependency('age_group', 'Result_of_Treatment', alpha)

True

**Вывод**: результат лечения не зависит от возраста.

### Тип бородавки
Проверим зависит ли результат лечения от типа бородавки.

In [6]:
alpha = 0.05
test_dependency('sex', 'Type', alpha)

True

**Вывод**: результат лечения не зависит от типа бородавки.

### Количество бородавок

Будем проверять гипотезу "среднее число борадавок одиноково при разных результатах лечения" против альтернативы "среднее число бородавок выше у тех, кому лечение не помогло" с помощью t-теста.

In [7]:
alpha = 0.05
stat, pval = ttest_ind(df[df.Result_of_Treatment == 0].Number_of_Warts, df[df.Result_of_Treatment == 1].Number_of_Warts)
pval > alpha

True

**Вывод**: среднее число бородавок одинаково при разных результатах лечения.

### Время до начала лечения

Будем проверять гипотезу "среднее время до начала лечения одиноково при разных результатах лечения" против альтернативы "среднее время до начала лечения больше у тех, кому лечение не помогло" с помощью t-теста.

In [8]:
alpha = 0.05
stat, pval = ttest_ind(df[df.Result_of_Treatment == 0].Time, df[df.Result_of_Treatment == 1].Time)
pval > alpha

False

**Вывод**: Гипотеза о равестве среднего временеи до начала лечения при разных результататах лечения может быть отвергнута.

### Размер бородавок
Будем проверять гипотезу "средний размер бородавок одиноков при разных результатах лечения" против альтернативы "средний размер бородавок больше у тех, кому лечение не помогло" с помощью t-теста.

In [9]:
alpha = 0.05
stat, pval = ttest_ind(df[df.Area == 0].Area, df[df.Area == 1].Area)
pval > alpha

False

**Вывод**: Гипотеза о равестве среднего размера бородавок при разных результататах лечения может быть отвергнута.

### Диаметр уплотнения
Будем проверять гипотезу "средний диаметр уплотнения одиноков при разных результатах лечения" против альтернативы "средний диаметр уплотнения больше у тех, кому лечение не помогло" с помощью t-теста.

In [10]:
alpha = 0.05
stat, pval = ttest_ind(df[df.Area == 0].induration_diameter, df[df.Area == 1].induration_diameter)
pval > alpha

False

**Вывод**: Гипотеза о равестве среднего диаметра уплотнения при разных результататах лечения может быть отвергнута.

### Корреляция
Результат лечиния зависит от время до начала лечения, размера бородавок и их диаметра уплотнения.
Посмотрим как между собой скоррелированы эти признаки.

In [11]:
cs = ['Time', 'Area', 'induration_diameter']
for c1 in cs:
    for c2 in cs:
        print(c1, '-', c2, ':', np.corrcoef(df[c1], df[c2])[0][1])

Time - Time : 0.9999999999999999
Time - Area : -0.04725351461748694
Time - induration_diameter : -0.183655918703116
Area - Time : -0.04725351461748693
Area - Area : 0.9999999999999998
Area - induration_diameter : 0.11926246662404304
induration_diameter - Time : -0.183655918703116
induration_diameter - Area : 0.11926246662404305
induration_diameter - induration_diameter : 1.0
